In [123]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [114]:
raw_data_path = '../data/raw/'

name = '17punto10'
reviews_raw = pd.read_csv(raw_data_path + 'collected_reviews_' + name + '.csv')
resumme_raw = pd.read_csv(raw_data_path + 'resumme_' + name + '.csv')
display(resumme_raw)
display(reviews_raw.sample(5))

,stars,reviews
0,5,571
1,4,191
2,3,35
3,2,13
4,1,13


,author,local_guide_info,rating,review,date_text,text_backup
8,Beatriz CD,Local Guide · 40 reseñas · 48 fotos,5 estrellas,"Fuimos de celebración,el sitio es pequeño, de ...",Hace 6 meses,Beatriz CD\nLocal Guide · 40 reseñas · 48 foto...
2,Silvia Carbonell,3 reseñas · 6 fotos,5 estrellas,Cenamos en familia y fue una experiencia gastr...,Hace un mes,Silvia Carbonell\n3 reseñas · 6 fotos\n\n\n...
18,Gema Pérez,Local Guide · 54 reseñas · 192 fotos,5 estrellas,Atención y comida maravillosa.\nNo sabría eleg...,Hace un año,Gema Pérez\nLocal Guide · 54 reseñas · 192 fot...
14,Sonia GARCIA,Local Guide · 227 reseñas · 928 fotos,4 estrellas,Bonito restaurante en el centro aunque a su ve...,Hace un año,Sonia GARCIA\nLocal Guide · 227 reseñas · 928 ...
25,isabel g. b.,Local Guide · 21 reseñas · 2 fotos,5 estrellas,El pasado sábado descubrimos este estupendo re...,Hace 3 meses,isabel g. b.\nLocal Guide · 21 reseñas · 2 fot...


In [116]:
raw_data_path = '../data/raw/'
reviews_raw = pd.read_csv(raw_data_path + 'collected_data.csv')
reviews_raw.drop(columns=['price', 'food', 'service', 'atmosphere', 'recommendations'], inplace=True)
reviews_raw.rename(columns = {'date':'date_text'}, inplace = True)
display(reviews_raw.sample(5))

,author,local_guide_info,rating,review,date_text,text_backup
8,jose maria jarillo,Local Guide · 159 reseñas · 515 fotos,5 estrellas,"Gran bar de tapas en la zona baja de chamberi,...",Hace 4 meses,jose maria jarillo\nLocal Guide · 159 reseñas ...
272,Marcela Ospina,18 reseñas · 2 fotos,5 estrellas,NaN,Hace un año,Marcela Ospina\n18 reseñas · 2 fotos\n\n\n\...
325,Fer Romero,Local Guide · 52 reseñas,5 estrellas,NaN,Hace 5 meses,Fer Romero\nLocal Guide · 52 reseñas\n\n\n\...
73,Filly I.,Local Guide · 9 reseñas · 94 fotos,1 estrella,Estafa .! Pedí una caña doble más un acuarius...,Hace 4 meses,Filly I.\nLocal Guide · 9 reseñas · 94 fotos\n...
53,Ignacio Vinuesa,Local Guide · 456 reseñas · 4.185 fotos,4 estrellas,Bar pequeño muy animado. Con mesas en terraza....,Hace 6 años,Ignacio Vinuesa\nLocal Guide · 456 reseñas · 4...


In [117]:
restaurant_search_words = {
    'service': r'Servicio\n([^\n]+)',
    'meal_type': r'Tipo de comida\n([^\n]+)',
    'price_per_person': r'Precio por persona\n([0-9€\- ]+)',
    'food_score': r'Comida: (\d+)',
    'service_score': r'Servicio: (\d+)',
    'atmosphere_score': r'Ambiente: (\d+)',
    'recommended': r'Platos recomendados\n([^\n]+)'
}

In [133]:
def extractRestaurantDetailsFromReview(sample, search_words=None, verbose=False):
    clean_text = re.sub(r'\\ue[0-9a-f]{3}', '', sample)
    clean_text = re.sub(r'\n+', '\n', clean_text)
    clean_text = clean_text.strip()

    # Store extracted values
    extracted_values = []

    # Loop through search words to extract values dynamically
    for key, regex in search_words.items():
        match = re.search(regex, clean_text)
        value = match.group(1) if match else ''
        extracted_values.append(value)

    return extracted_values

def applyExtractDetails(df, search_words=None):
    column_names = list(search_words.keys())
    df[column_names] = df['text_backup'].apply(lambda x: pd.Series(extractRestaurantDetailsFromReview(x, search_words=search_words)))
    return df

def extractReviewCount(text):
    if isinstance(text, str):  # Verify if its a string
        match = re.search(r'(\d+)\s+reseñas', text)
        if match:
            return int(match.group(1))
    return None

def extractStarRating(text):
    match = re.search(r'(\d+)\s+estrellas', text)
    if match:
        return int(match.group(1))
    return None

def extractRecommendations(recommendations):
    recommendations_list = recommendations.split(', ')
    if ' y ' in recommendations_list[-1]:
        last_dishes = recommendations_list[-1].rsplit(' y ', 1)
        recommendations_list = recommendations_list[:-1] + last_dishes
    return recommendations_list

def convertToDate(date_text):
    today = datetime.today()

    if 'semana' in date_text:
        # Extract number of weeks, default to 1 if no number is present
        weeks = pd.Series(date_text).str.extract(r'(\d+)')[0]
        weeks = int(weeks.iloc[0]) if pd.notna(weeks.iloc[0]) else 1
        monday_of_current_week = today - timedelta(days=today.weekday())  # Get Monday of the current week
        return monday_of_current_week.date() - timedelta(weeks=weeks)

    elif 'mes' in date_text:
        # Extract number of months, default to 1 if no number is present
        months = pd.Series(date_text).str.extract(r'(\d+)')[0]
        months = int(months.iloc[0]) if pd.notna(months.iloc[0]) else 1
        target_date = today - relativedelta(months=months)
        # Return the first day of the target month
        return target_date.replace(day=1).date()

    elif 'año' in date_text:
        # Extract number of years, default to 1 if no number is present
        years = pd.Series(date_text).str.extract(r'(\d+)')[0]
        years = int(years.iloc[0]) if pd.notna(years.iloc[0]) else 1
        target_date = today - relativedelta(years=years)
        # Return the first day of the target year
        return target_date.replace(month=1, day=1).date()

    return None  # Return None if no match is found


In [136]:
reviews = reviews_raw.copy()

reviews['local_guide_reviews'] = reviews['local_guide_info'].apply(extractReviewCount)
reviews['rating_score'] = reviews['rating'].apply(extractStarRating)
reviews = applyExtractDetails(reviews, search_words = restaurant_search_words)
reviews['recommendations_list'] = reviews['recommended'].apply(extractRecommendations)
reviews['date'] = reviews['date_text'].apply(convertToDate)
reviews['food_score'] = reviews['food_score'].apply(pd.to_numeric, errors='coerce')
reviews['service_score'] = reviews['service_score'].apply(pd.to_numeric, errors='coerce')
reviews['atmosphere_score'] = reviews['atmosphere_score'].apply(pd.to_numeric, errors='coerce')

reviews.drop(columns = ['text_backup', 'local_guide_info', 'rating', 'author', 'recommended', 'date_text'], inplace = True)
reviews.reset_index(inplace=True)
reviews.rename(columns={'index': 'review_id'}, inplace=True)

In [137]:
# Fill NA values
reviews['local_guide_reviews'] = reviews['local_guide_reviews'].fillna(1)
reviews['rating_score'] = reviews['rating_score'].fillna(1)

In [138]:
display(reviews.head(30))


,review_id,review,local_guide_reviews,rating_score,service,meal_type,price_per_person,food_score,service_score,atmosphere_score,recommendations_list,date
0,0,Fantástico ambiente. Camareros rápidos y amabl...,99.0,5.0,,,1-10 €,5.0,5.0,5.0,"[Tapas Y Dobles de Cerveza, Patatas Bravas O A...",2024-09-09
1,1,El Bar Sierra en Madrid es un verdadero tesoro...,35.0,5.0,,,1-10 €,5.0,5.0,5.0,[Tapas Y Dobles de Cerveza],2024-07-01
2,2,Tapas absolutamente de locos. Por apenas 3.30 ...,198.0,5.0,,,,5.0,5.0,5.0,"[Tapas Y Dobles de Cerveza, Patatas Bravas O A...",2024-04-01
3,3,La leche!! Este sitio es el mejor con diferenc...,84.0,5.0,,,,5.0,5.0,5.0,[Tapas Y Dobles de Cerveza],2024-09-09
4,4,"Por el precio nos gustó mucho, los tamaños de ...",308.0,5.0,,,,5.0,3.0,3.0,[],2024-03-01
5,5,En la zona de Chamberí en Madrid. La estación ...,278.0,5.0,,,1-10 €,5.0,4.0,4.0,[],2024-07-01
6,6,Os cuento según mi opinión:\n\nContras:\nSitio...,61.0,4.0,,,1-10 €,4.0,3.0,4.0,[],2024-06-01
7,7,"El camarero mu majo, el baño limpio y las tapa...",69.0,5.0,,,1-10 €,5.0,5.0,5.0,"[Patatas Bravas O Alioli, Tapa de Macarrones]",2024-08-01
8,8,"Gran bar de tapas en la zona baja de chamberi,...",159.0,5.0,,,10-20 €,5.0,5.0,5.0,"[Tapas Y Dobles de Cerveza, Patatas Bravas O A...",2024-05-01
9,9,Hemos comido totalmente con dos consumiciones ...,323.0,5.0,,,1-10 €,4.0,5.0,5.0,[Tapas Y Dobles de Cerveza],2024-06-01
